In [1]:
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, GRU, Dense
from tensorflow.keras.optimizers import Adam

file_path = '/kaggle/input/freezing-of-gait/FOG/freezing-of-gait-exploration-main/freezing-of-gait-exploration-main/sliding_window_dataset'
file_names = [
    'S01R01experiment0', 'S01R02experiment0',
    'S02R01experiment0', 'S02R02experiment0',
    'S03R01experiment0', 'S03R01experiment1', 'S03R02experiment0', 'S03R03experiment0',
    'S04R01experiment0',
    'S05R01experiment0', 'S05R01experiment1', 'S05R01experiment2', 'S05R01experiment3',
    'S06R01experiment0',
]
file_name = [
    'S07R01experiment0', 'S07R02experiment0',
    'S08R01experiment0', 'S08R01experiment1', 'S08R01experiment2', 'S08R01experiment3',
    'S09R01experiment0', 'S09R01experiment1'
]
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
strategy = tpu_strategy

def create_model():
    # Define your model architecture
    model = Sequential()
    model.add(InputLayer((1000, 4)))
    model.add(GRU(64))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

def train_model(X_train, y_train, X_test, y_test, initial_model=None):
    with strategy.scope():
        if initial_model is None:
            model = create_model()
        else:
            model = initial_model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=256)

    return model

# Load and preprocess your data
initial_model = None
for name in file_names:
    X = np.load(f'{file_path}/{name}_X.npy')
    y = np.load(f'{file_path}/{name}_y.npy')
    y = np.array([[i] for i in y])
    length = X.shape[0]
    test_data_idx_start = int(length * 0.8)
    X_train = X[:test_data_idx_start]
    y_train = y[:test_data_idx_start]
    X_test = X[test_data_idx_start:]
    y_test = y[test_data_idx_start:]
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

    # Train the model within the TPU strategy scope
    with strategy.scope():
        initial_model = train_model(X_train, y_train, X_test, y_test, initial_model)

# Save the final trained model
initial_model.save('/kaggle/working/gru_model.h5')


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

200/200 [==============================] - 24s 99ms/step - loss: 0.0069 - accuracy: 0.9988 - val_loss: 0.0188 - val_accuracy: 0.9977
Epoch 2/100
200/200 [==============================] - 16s 79ms/step - loss: 0.0069 - accuracy: 0.9987 - val_loss: 0.0168 - val_accuracy: 0.9978
Epoch 3/100
200/200 [==============================] - 16s 78ms/step - loss: 0.0065 - accuracy: 0.9988 - val_loss: 0.0177 - val_accuracy: 0.9976
Epoch 4/100
200/200 [==============================] - 16s 80ms/step - loss: 0.0072 - accuracy: 0.9986 - val_loss: 0.0225 - val_accuracy: 0.9964
Epoch 5/100
200/200 [==============================] - 16s 79ms/step - loss: 0.0064 - accuracy: 0.9989 - val_loss: 0.0191 - val_accuracy: 0.9975
Epoch 6/100
200/200 [==============================] - 16s 78ms/step - loss: 0.0063 - accuracy: 0.9988 - val_loss: 0.0182 - val_accuracy: 0.9977
Epoch 7/100
200/200 [==============================] - 16s 79ms/step - loss: 0.0062 - accuracy: 0.9988 - val_loss: 0.0248 - val_accuracy: 0.99

2023-07-18 13:07:35.635660: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-07-18 13:07:35.784570: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


157/157 [==============================] - ETA: 0s - loss: 0.0061 - accuracy: 0.9991

2023-07-18 13:07:51.046337: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-07-18 13:07:51.162168: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


157/157 [==============================] - 21s 105ms/step - loss: 0.0061 - accuracy: 0.9991 - val_loss: 6.8176e-05 - val_accuracy: 1.0000
Epoch 2/100
157/157 [==============================] - 12s 79ms/step - loss: 0.0038 - accuracy: 0.9995 - val_loss: 5.1372e-05 - val_accuracy: 1.0000
Epoch 3/100
157/157 [==============================] - 12s 79ms/step - loss: 0.0035 - accuracy: 0.9996 - val_loss: 1.0888e-04 - val_accuracy: 1.0000
Epoch 4/100
157/157 [==============================] - 12s 79ms/step - loss: 0.0033 - accuracy: 0.9996 - val_loss: 4.2553e-05 - val_accuracy: 1.0000
Epoch 5/100
157/157 [==============================] - 12s 79ms/step - loss: 0.0032 - accuracy: 0.9996 - val_loss: 2.1498e-05 - val_accuracy: 1.0000
Epoch 6/100
157/157 [==============================] - 14s 87ms/step - loss: 0.0033 - accuracy: 0.9995 - val_loss: 5.0366e-05 - val_accuracy: 1.0000
Epoch 7/100
157/157 [==============================] - 13s 81ms/step - loss: 0.0033 - accuracy: 0.9995 - val_loss: 1.

In [2]:
for name in file_name:
    X1 = np.load(f'{file_path}/{name}_X.npy')
    length = X1.shape[0]
    y1 = np.load(f'{file_path}/{name}_y.npy')
    y1 = np.array([[i] for i in y1])
    test_data_idx_start = int(length)
    X1_test = X1[:test_data_idx_start]
    y1_test = y1[:test_data_idx_start]

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn import metrics

# Load the saved model
model = load_model('/kaggle/working/gru_model.h5')

yhat = model.predict(X1_test)# metrics# metrics
from sklearn import metrics
TN, FP, FN, TP = metrics.confusion_matrix(y1_test, np.array([0 if y < .5 else 1 for y in yhat])).ravel()


print('Sensitivity, hit rate, recall, or true positive rate: ')
TPR = TP/(TP+FN)
print(TPR)
print('Specificity or true negative rate')
TNR = TN/(TN+FP) 
print(TNR)
print('Precision or positive predictive value')
PPV = TP/(TP+FP)
print(PPV)
print('Negative predictive value')
NPV = TN/(TN+FN)
print(NPV)
print('Fall out or false positive rate')
FPR = FP/(FP+TN)
print(FPR)
print('False negative rate')
FNR = FN/(TP+FN)
print(FNR)
print('False discovery rate')
FDR = FP/(TP+FP)
print(FDR)

print('Overall accuracy')
ACC = (TP+TN)/(TP+FP+FN+TN)
print(ACC)

578/578 [==============================] - 59s 101ms/step
Sensitivity, hit rate, recall, or true positive rate: 
0.998582995951417
Specificity or true negative rate
0.857533234859675
Precision or positive predictive value
0.7188866219761003
Negative predictive value
0.9993974866586331
Fall out or false positive rate
0.14246676514032497
False negative rate
0.001417004048582996
False discovery rate
0.28111337802389974
Overall accuracy
0.8952380952380953
